# Grouping

In [ ]:
import pandas as pd
import numpy as np

data = {
    '주문번호': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010],
    '고객ID': ['A', 'B', 'A', 'C', 'B', 'A', 'D', 'C', 'B', 'D'],
    '상품카테고리': ['전자제품', '의류', '가구', '전자제품', '의류', '식품', '가구', '식품', '전자제품', '의류'],
    '구매액': [150000, 75000, 220000, 95000, 82000, 45000, 180000, 35000, 120000, 62000],
    '배송지역': ['서울', '부산', '서울', '인천', '서울', '부산', '인천', '서울', '부산', '인천'],
    '할인률': [0.05, 0.1, 0, 0.2, 0.1, 0, 0.05, 0.15, 0.2, 0]
}

df = pd.DataFrame(data)

In [ ]:
df.groupby('고객ID')['구매액'].sum()

# 고객 id로 그룹핑
id_group = df.groupby('고객ID')

# 그룹 확인 (고객 ID들 그룹) !!!!!!!!!!!!!!!!!!!!!!!!
id_group.groups.keys()
print(id_group.groups) # .groups는 그룹핑 결과를 '딕셔너리 형태'로 보여줌

# 특정 그룹 데이터 확인
id_group.get_group('A')



# 여러 col으로 그룹핑 -> 멀티 인덱스를 가진 Series임.. 사실 값은 '구매액' 행 부분밖에 없어~!
multi_group = df.groupby(['고객ID', '상품카테고리'])['구매액'].sum()
print(multi_group)
print(multi_group.unstack())

# DF로 변환
multi_group.to_frame()
multi_group.to_frame().index[0]



In [ ]:
# 1개의 col(구매액)에 집계함수 여러개 적용
df.groupby('고객ID')['구매액'].agg(['sum', 'mean', 'count'])


# n개의 col에 m개 집계함수 적용
df.groupby('고객ID').agg({
    '구매액': ['sum', 'mean'],
    '할인률': ['mean', 'max']
})




# 사용자 정의 집계함수 (커스텀 가능) ???????????
def discount_amount(price):
    return (price * df.loc[price.index, '할인률']).sum()

df.groupby('고객ID')['구매액'].agg([
    # AS로 이름 붙여주기, function
    ('총구매액', 'sum'),
    ('평균구매액', 'mean'),
    ('할인총액', discount_amount)
])

In [ ]:
# 집계 함수 응용
import pandas as pd
import numpy as np

# 샘플 데이터
df = pd.DataFrame({
    '상품ID': ['A001', 'A002', 'A001', 'A003', 'A002', 'A004', 'A003', 'A001', 'A002', 'A004'],
    '판매일자': pd.date_range('2023-01-01', periods=10),
    '판매수량': [5, 3, 7, 2, 4, 6, 3, 8, 5, 4],
    '판매금액': [50000, 30000, 70000, 25000, 40000, 65000, 30000, 80000, 50000, 45000],
    '반품수량': [0, 1, 0, 0, 0, 2, 1, 0, 0, 1],
    '고객평점': [4.5, 3.8, 4.2, 5.0, 4.0, 3.5, 4.2, 4.8, 3.9, 4.1]
})

df.sort_values('상품ID')

In [ ]:
df.groupby('상품ID').agg({
    '판매수량': [('총합', 'sum'), 'mean', 'count'],
    '판매금액': ['sum', 'mean'],
    '반품수량': ['sum'],
    '고객평점': ['mean']
})

In [ ]:
# shift + enter 누르면 셀 생김

In [ ]:
# 커스텀 함수
# 총 판매수량 대비 반품수량 비율 계산

def return_rate(x):
    total_sold = df.loc[x.index, '판매수량'].sum()
    total_returned = df.loc[x.index, '반품수량'].sum()
    return total_returned / total_sold if total_sold > 0 else 0

df.groupby('상품ID').agg({
    '판매수량': ['sum', 'count'],
    '반품수량': ['sum', return_rate]
    })

In [ ]:
## 그룹별 순위 및 누적 계산
import pandas as pd
import numpy as np

# 샘플 데이터: 부서별 직원 실적
data = {
    '직원ID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112],
    '이름': ['김철수', '이영희', '박민수', '정지영', '최동민', '강준호', '윤서연', '임태혁', '한미래', '송지원', '오민지', '홍길동'],
    '부서': ['영업', '개발', '영업', '인사', '개발', '영업', '개발', '인사', '영업', '개발', '영업', '인사'],
    '월별실적': [120, 85, 95, 110, 75, 135, 95, 105, 115, 90, 125, 100],
    '고객평가': [4.5, 3.8, 4.2, 4.7, 3.9, 4.8, 4.1, 4.3, 4.5, 4.0, 4.6, 4.2]
}

df = pd.DataFrame(data)
print("부서별 직원 실적 데이터:")
print(df)

In [ ]:
# 그룹 내 순위를 계싼해보자
dept_group = df.groupby('부서')

# 부서별 월별 실적! dense는 랭크 방식! 윈도우 함수에서 배움!
df['부서순위_실적'] = dept_group['월별실적'].rank(method='dense', ascending=False)

df

In [ ]:
# 누적 합계 및 누적 통계
# 부서별 누적 실적 합계 -> accmulate cummulate

df['부서별누적합계'] = df.groupby('부서')['월별실적'].cumsum()
df['부서별누적최대'] = df.groupby('부서')['월별실적'].cummax()

# 그룹별 비율 계산
# 부서별 총 실적 대비, 개인 실적 비율
df['부서총실적'] = df.groupby('부서')['월별실적'].transform('sum') # transform
df['부서기여도'] = df['월별실적'] / df['부서총실적']


# 복합응용
# 성과점수 = 0.7 실적 + 0.3 (평가*20)
df['성과점수'] = df['월별실적']*0.7 + df['고객평가']* 0.3 * 20
df['부서순위_성과'] = dept_group['성과점수'].rank(method='dense', ascending=False) # 이 부분 다시!!!!!!!!!!!!!!!!!!


df

# 실습하깅: 매출 데이터 그룹별 분석

In [ ]:
# 실습: 매출 데이터 그룹별 분석
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 매출 데이터 생성
np.random.seed(42)

# 날짜 생성 (2023년 전체)
dates = pd.date_range('2023-01-01', '2023-12-31')
n_records = 500

data = {
    '주문ID': np.arange(1001, 1001 + n_records),
    '주문일자': np.random.choice(dates, n_records),
    '고객ID': np.random.choice([f'CUST{i:03d}' for i in range(1, 101)], n_records),
    '상품ID': np.random.choice([f'PROD{i:03d}' for i in range(1, 51)], n_records),
    '카테고리': np.random.choice(['전자제품', '의류', '가구', '식품', '화장품', '도서', '스포츠'], n_records),
    '매출액': np.random.randint(10000, 500000, n_records),
    '수량': np.random.randint(1, 10, n_records),
    '지역': np.random.choice(['서울', '부산', '인천', '대구', '광주', '대전', '울산', '경기', '강원'], n_records),
    '결제방법': np.random.choice(['신용카드', '현금', '체크카드', '휴대폰', '계좌이체'], n_records),
    '고객등급': np.random.choice(['일반', '실버', '골드', 'VIP'], n_records)
}

df = pd.DataFrame(data)
df

In [ ]:
# 검색후 하기
# 날짜 정보 추출 -> 컬럼 추가 ['주문년월', '요일', '주']
# 'dt'는 datetime 속성에 접근하는 Pandas 전용 도우미!!!!!

df['주문년월'] =  df['주문일자'].dt.strftime('%Y-%m') #str format time

df['요일'] = df['주문일자'].dt.day_name()

df['주'] = df['주문일자'].dt.isocalendar().week
df.head()

# 특정 조건. 1/1인 행들만 확인
df[ df['주문일자'] =='2023-01-01' ]

In [ ]:
# 단가 계산 
# 단가 컬럼 추가 (매출액/수량) 
df['제품단가'] = (df['매출액'] / df['수량']).round(2) # round 둘째자리까지 남기고 반올림
df.head()

In [ ]:
# 카테고리별 매출 분석 
# 매출액 총합,평균,개수 /  수량 총합

df.groupby('카테고리').agg({
    '매출액': ['sum', 'mean', 'count'],
    '수량': 'sum'
})

In [ ]:
# 월별 매출 트렌드
# '주문년월' 컬럼으로 매출액 sum, 주문ID count, 단가 mean

df.groupby('주문년월').agg({
    '매출액': 'sum',
    '주문ID': 'count',
    '제품단가':  lambda x: x.mean().round(2) # 여기 다시 한번 보기 ????????????????????
}).rename(columns={'주문ID':'주문건수'}) # 컬럼 이름 바꾸장

In [ ]:
# 지역 & 카테고리별 매출 분석
# unstack -> 다중인덱스 구조를 열로 펼쳐서 재구조화 함!!!!!
regional_df = df.groupby(['지역', '카테고리'])['매출액'].sum().unstack()

In [ ]:
# 위의 DF을 히트맵으로 시각화!
plt.figure(figsize=(12, 8))
sns.heatmap(regional_df, cmap='YlGnBu', annot=True, fmt='.0f')
plt.title('지역별, 카테고리별 매출액 히트맵')
plt.tight_layout()
plt.show()

In [ ]:
# 요일별 고객 등급별 매출 패턴

# 요일의 순서 지정!!! 날짜에 순서 부여해서 범주화!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['요일'] = pd.Categorical(df['요일'], categories=day_order, ordered=True)


# 가설은 있다 치고 검증 중인 거임~~~~~
result = df.groupby(['요일', '고객등급']).agg({
    '매출액': ['sum', 'mean', 'count']
})

# 요일별로, 매출 총액 기준 정렬 # (매출액, sum)은 agg만들 때 애초에 튜플 형태!라서 쓸 때도 그렇게
result.sort_values(['요일', ('매출액', 'sum')], ascending=False)

In [ ]:
# 결제방법별 분석 및 고객 행동 (매출액-sum mean count, 단가-mean)

def ratio(x):
    return x.sum() / df['매출액'].sum()

df.groupby('결제방법').agg({
    '매출액': ['sum', 'mean', 'count', ratio],
    '제품단가': 'mean'
})


In [ ]:
# 고객id 별 구매 패턴 (매출액 sum mean count, 구매한 상품 고유 수, 고유 카테고리 )
# 고유한게 몇개인지? nunique, 고유한게 뭔지? unique
df.groupby('고객ID').agg({
    '매출액':['sum', 'mean', 'count'],
    '상품ID': 'nunique',
    '카테고리': 'unique'
}).sort_values[([(), ()])]

In [ ]:
# 주별 매출 추이
df[['주', '매출액']].head()
# 일단 주 단위로 정렬을 하고? 그 다음 diff...?
# 아니면 주별로 매출총액을 구하고, 그 다음 주 단위로 정렬..
weekly_sales = df.groupby('주')['매출액'].sum().reset_index()
weekly_sales = weekly_sales.sort_values(['주문년월', '주'])

# .pct_change()는 바로 이전 행과의 비율 차이를 계산
# fillna(0)으로 첫 주의 결측치는 0으로 처리 (전주 없으니까 0으로)
weekly_sales['매출증감률'] = weekly_sales['매출액'].pct_change().fillna(0)

weekly_sales



In [ ]:
# 매출 TOP 10 상품 + 카테고리별로 보고 싶었음
df.groupby(['카테고리', '상품ID'])['매출액'].sum().sort_values(ascending=False).head(10)

# 다 보지 말고 딱 10개만. 효율적임!
df.groupby(['카테고리', '상품ID'])['매출액'].sum().nlargest(10)

# 데이터 결합
1. 데이터 단순 결합(행 결합)
2. 데이터 병합
3. Index 기준 Join

In [ ]:
# 데이터 결합 기초 (concat) - 행결합 (Col 똑같아야 함)

# 샘플 데이터 생성
# 첫 번째 데이터프레임: 1월 판매 데이터
df1 = pd.DataFrame({
    '상품ID': ['A001', 'A002', 'A003', 'A004', 'A005'],
    '상품명': ['노트북', '스마트폰', '태블릿', '헤드폰', '스피커'],
    '판매량_1월': [10, 20, 15, 30, 25]
})

# 두 번째 데이터프레임: 2월 판매 데이터
df2 = pd.DataFrame({
    '상품ID': ['A001', 'A003', 'A005', 'A006', 'A007'],
    '상품명': ['노트북', '태블릿', '스피커', '마우스', '키보드'],
    '판매량_2월': [12, 18, 23, 15, 19]
})

# 기본 concat - 행 결합 -> 안맞는 컬럼은 NaN
pd.concat([df1, df2])

# 인덱스 초기화 (겹치는 인덱스 없이 처음부터 다시)
pd.concat([df1, df2], ignore_index=True)

# 열 방향 결합
pd.concat([df1, df2], axis=1)

# join inner (공통 열만 유지)
pd.concat([df1, df2], join='inner')

In [ ]:
# 데이터 병합 (merge)
import pandas as pd
import numpy as np

# 샘플 데이터 생성
# 상품 정보 데이터프레임
products = pd.DataFrame({
    '상품ID': ['P001', 'P002', 'P003', 'P004', 'P005'],
    '상품명': ['노트북', '스마트폰', '태블릿', '헤드폰', '스피커'],
    '가격': [1200000, 850000, 500000, 150000, 75000],
    '카테고리': ['컴퓨터', '모바일', '모바일', '음향기기', '음향기기']
})

# 주문 정보 데이터프레임
orders = pd.DataFrame({
    '주문번호': [1001, 1002, 1003, 1004, 1005, 1006],
    '고객ID': ['C001', 'C002', 'C003', 'C001', 'C004', 'C002'],
    '상품ID': ['P001', 'P002', 'P003', 'P002', 'P005', 'P006'],
    '수량': [1, 2, 1, 1, 3, 2],
    '주문일자': ['2023-01-05', '2023-01-10', '2023-01-15', '2023-01-20', '2023-01-25', '2023-01-30']
})

print("상품 정보:")
print(products)
print("\n주문 정보:")

In [ ]:
customers = pd.DataFrame({
    'ID': ['C001', 'C002', 'C003', 'C004', 'C005'],
    '이름': ['김철수', '이영희', '박민수', '정지영', '최동민'],
    '등급': ['VIP', '골드', '실버', '골드', '브론즈']
})

# 열 이름이 다르면?
pd.merge(
    orders, 
    customers, 
    left_on='고객ID',  # orders 데이터프레임의 열 이름
    right_on='ID',    # customers 데이터프레임의 열 이름
    how='inner'
)

## 해볼테면 해봐

In [ ]:
import pandas as pd
import numpy as np

# 샘플 데이터셋 생성
# 1. 고객 정보 데이터
customers = pd.DataFrame({
    '고객ID': [f'CUST{i:03d}' for i in range(1, 11)],
    '이름': ['김철수', '이영희', '박민수', '정지영', '최동민', '강준호', '윤서연', '임태혁', '한미래', '송지원'],
    '성별': ['남', '여', '남', '여', '남', '남', '여', '남', '여', '여'],
    '연령대': ['30대', '20대', '40대', '30대', '50대', '20대', '40대', '30대', '20대', '50대'],
    '가입일자': pd.date_range('2023-01-01', periods=10, freq='3D'),
    '지역': ['서울', '부산', '서울', '인천', '대구', '서울', '부산', '인천', '서울', '대구']
})

# 2. 주문 정보 데이터
np.random.seed(42)
n_orders = 50

orders = pd.DataFrame({
    '주문번호': [f'ORD{i:04d}' for i in range(1, n_orders+1)],
    '고객ID': np.random.choice(customers['고객ID'], n_orders),
    '주문일자': pd.date_range('2023-01-05', periods=n_orders, freq='2D'),
    '결제방법': np.random.choice(['신용카드', '체크카드', '계좌이체', '간편결제'], n_orders),
    '배송상태': np.random.choice(['배송완료', '배송중', '주문확인', '배송지연'], n_orders, p=[0.7, 0.15, 0.1, 0.05])
})

# 3. 주문 상세 정보 데이터
n_details = 80
products = ['노트북', '스마트폰', '태블릿', '헤드폰', '스피커', '키보드', '마우스', '모니터']
categories = ['전자제품', '컴퓨터', '주변기기', '음향기기']

order_details = pd.DataFrame({
    '상세번호': [f'ITEM{i:04d}' for i in range(1, n_details+1)],
    '주문번호': np.random.choice(orders['주문번호'], n_details),
    '상품명': np.random.choice(products, n_details),
    '카테고리': np.random.choice(categories, n_details),
    '수량': np.random.randint(1, 5, n_details),
    '가격': np.random.choice([50000, 100000, 150000, 800000, 1200000, 1500000], n_details),
    '할인율': np.random.choice([0, 0.1, 0.2, 0.3], n_details)
})

# 4. 배송 정보 데이터
shipping = pd.DataFrame({
    '주문번호': orders['주문번호'].unique(),
    '배송사': np.random.choice(['A택배', 'B물류', 'C익스프레스'], len(orders['주문번호'].unique())),
    '배송비': np.random.choice([0, 2500, 5000], len(orders['주문번호'].unique())),
    '출고일자': pd.date_range('2023-01-06', periods=len(orders['주문번호'].unique()), freq='2D')
})

# 5. 고객 만족도 데이터 (일부 주문에 대해서만)
satisfaction_orders = np.random.choice(orders['주문번호'], size=30, replace=False)
satisfaction = pd.DataFrame({
    '주문번호': satisfaction_orders,
    '만족도': np.random.randint(1, 6, 30),
    '리뷰': np.random.choice(['긍정', '중립', '부정'], 30, p=[0.6, 0.3, 0.1]),
    '리뷰일자': pd.date_range('2023-01-15', periods=30, freq='3D')
})

In [ ]:
# 주문- 주문상세 테이블 병합!!
# 합친다- 테이블 a, 테이블 b
pd.merge(orders, order_details, on='주문번호', how='inner').head()

# 테이블 a에 합친다- 테이블 b
orders.merge(order_details, on='주문번호', how='inner').head()

In [ ]:
## order_details에서 총금액 뽑고, 주문번호로 grouping, orders에 주문금액 col 추가하기

# 1. 총 주문금액 계산
total_series = order_details['수량'] * order_details['가격'] * (1-order_details['할인율'])

order_details['총금액'] = total_series
order_details.head()

In [ ]:
# 2. 주문별 합계 계산 -> 테이블 생성
order_totals = order_details.groupby('주문번호')['총금액'].sum().reset_index() # 주문번호가 인덱스인 시리즈에서, reset해서 df로 만듦
order_totals.columns

In [ ]:
# 3. orders와 order_totals 결합 -> 원래 없던 컬럼을 추가했슨
order_with_total = orders.merge(order_totals, on='주문번호', how='left')

In [ ]:
order_details[order_details.duplicated(subset=['주문번호'], keep=False)].sort_values('주문번호')

In [ ]:
# 주문 데이터에 사용자 병합
orders_customers = order_with_total.merge(customers, on='고객ID')
orders_customers.head()

In [ ]:
# 배송 정보 추가
full_orders = orders_customers.merge(shipping, on='주문번호')
full_orders.head()

# 배송 소요일 추가해보장~~ days라고 나오는 이유는 타입이 date라서. 근데 timedelta -> int로 변환하고 싶음
full_orders['배송소요일'] = (full_orders['출고일자'] - full_orders['주문일자']).dt.days
full_orders.head()

In [ ]:
# 만족도 정보 결합
with_satisfaction = full_orders.merge(satisfaction, on='주문번호', how='left')
with_satisfaction.head()

In [ ]:
# 분석: 연령대별, 성별 총 주문 금액
spending_stat = with_satisfaction.groupby(['연령대', '성별'])['총금액'].agg(['sum', 'mean','count'])
spending_stat

In [ ]:
# 분석: 결제 방법별 평균 주문금액 및 건수
payment_analysis = with_satisfaction.groupby('결제방법').agg({
    '총금액':['mean', 'sum', 'count']
}).reset_index()

payment_analysis

In [ ]:
# 분석: 만족도와 총 주문금액 관계

# 결측지 제거
order_sati = with_satisfaction.dropna(subset=['만족도'])

# 만족도별 평균 주문 금액
sati_spend = order_sati.groupby('만족도')['총금액'].mean().reset_index()
sati_spend

In [ ]:
# 지역별 배송사 선호
region_shipping = with_satisfaction.groupby(['지역', '배송사']).size().unstack().fillna(0)
region_shipping

# 이상치(outlier)
## 이상치 탐지

In [78]:
np.random.seed(42)
n = 1000

# 정상적인 데이터 생성
normal_prices = np.random.normal(50000, 15000, 900)  # 정상 가격대
normal_prices = np.round(normal_prices).astype(int)  # 정수 가격

normal_quantities = np.random.poisson(3, 900) + 1   # 정상 수량
normal_quantities = normal_quantities.astype(int)    # 정수 수량

# 이상치 데이터 추가
outlier_prices = np.random.uniform(200000, 500000, 100)  # 이상 고가 상품
outlier_prices = np.round(outlier_prices).astype(int)

outlier_quantities = np.random.uniform(50, 100, 100)      # 이상 대량 주문
outlier_quantities = np.round(outlier_quantities).astype(int)

# 전체 데이터 결합
prices = np.concatenate([normal_prices, outlier_prices])
quantities = np.concatenate([normal_quantities, outlier_quantities])

# DataFrame 생성
data = {
    '주문번호': [f'ORD{i:04d}' for i in range(1, n+1)],
    '가격': prices,
    '수량': quantities,
    '카테고리': np.random.choice(['전자제품', '의류', '가구', '식품', '도서'], n),
    '지역': np.random.choice(['서울', '부산', '인천', '대구', '광주'], n)
}

df = pd.DataFrame(data)
df['총금액'] = df['가격'] * df['수량']

df.head()

,주문번호,가격,수량,카테고리,지역,총금액
0,ORD0001,57451,5,가구,부산,287255
1,ORD0002,47926,5,식품,서울,239630
2,ORD0003,59715,4,식품,부산,238860
3,ORD0004,72845,7,도서,부산,509915
4,ORD0005,46488,3,식품,광주,139464


In [ ]:
%pip install -q scipy

In [ ]:
# 1. Z-score(표준점수)

# 가격 기준 이상치 탐지
prices_series = df['가격']
# Z-score 직접 계산하기? 각각의 값- 값들의 평균 / 표준편차
(prices_series- prices_series.mean()) / prices_series.std()

def detect_outlier_zscore(data_series, thershold=3.0): #thershold는 기준값. z-score의 임계값!!
    from scipy import stats # scipy 사용

    z_scores = stats.zscore(data_series)
    return np.abs(z_scores) > thershold # 3보다 큰 게 이상한 애들 mask!


# 이상치 데이터만 보기
detect_outlier_zscore(df['가격'], 2.0) # <- 이상치 기준 바꿀 수 있음

In [ ]:
# 2. IQR (InterQuartile Range 4분위) 방법

# 25, 50, 75%
#prices_series.describe()

def detect_outlier_iqr(data_series):
    Q1 = data_series.quantile(0.25)
    Q3 = data_series.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - (1.5*IQR)
    upper_bound = Q3 + (1.5*IQR)

    return (data_series < lower_bound) | (data_series > upper_bound)

df[detect_outlier_iqr(df['가격'])]

In [ ]:
# 3. 백분위수 방법

def detcet_outlier_perc(data_series, lower=1, upper=99):
    lower_bound = data_series.quantile(lower/100) # 하위 1%
    upper_bound = data_series.quantile(upper/100) # 상위 1%

    return (data_series < lower_bound) | (data_series > upper_bound)

odf = df[detcet_outlier_perc(df['가격'], lower=1, upper=97)] #IQR과 달리, 위아래 이상치 직접 조정 가능
odf.count()

## 이상치 처리

In [ ]:
df.head()

In [ ]:
# data에서 col에 IQR 기준 이상치를 삭제하는 함수
def remove_outliers_iqr(data: pd.DataFrame, col:str) -> pd.DataFrame:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (1.5*IQR)
    upperr_bound = Q3 + (1.5*IQR)
    mask = data[ (data[col] >= lower_bound) & (data[col] <= upperr_bound)]

    return data[mask]

df_rm1 = remove_outliers_iqr(df, '가격')

# 원본 길이, 정제 길이
print(len(df), len(df_rm1))



In [82]:
# 이상치 대체(Imputation) 하기

# 중앙값 대체
def replace_outliers_with_median(data: pd.DataFrame, col:str) -> pd.DataFrame:
    """IQR 이상치를 중앙값으로 대체"""
    data_replaced = data.copy()
    # IQR 이상치 탐지
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (1.5*IQR)
    upperr_bound = Q3 + (1.5*IQR)

    # 중앙값
    med_val = data[col].median()
    outlier_mask = ( data[col] <= lower_bound) | (data[col] >= upperr_bound)
    data_replaced.loc[outlier_mask, col] = int(med_val)

    return data_replaced

df_replaced = replace_outliers_with_median(df, '가격')
df_replaced['총금액'] = df_replaced['가격'] * df_replaced['수량']

df_replaced.describe()

,가격,수량,총금액
count,1000.000000,1000.000000,1.000000e+03
mean,50469.041000,10.992000,5.666339e+05
std,13765.080566,21.618448,1.133430e+06
min,9547.000000,1.000000,9.547000e+03
25%,41428.000000,3.000000,1.276982e+05
50%,52255.000000,4.000000,1.954185e+05
75%,58770.000000,6.000000,2.960168e+05
max,96183.000000,99.000000,5.173245e+06


In [ ]:
# 이상치 변환(Transform) 하기


# winsorization (윈저화) -> 이상치를 근처 값으로 바꾸기 !!!!!!!!!!

def winsorize_outliers(data, col, lower=5, upper=95):
    """윈저화: 극값을 특정 백분위 값으로 대체하겠다"""
    lower_bound = data[col].quantile(lower / 100)
    upper_bound = data[col].quantile(upper / 100)

    data_winsorized = data.copy()
    
    # clip: lower보다 작은 수를 다 인자로 바꾸고, upper보다 큰 수도 모두 인자로 바꾼다
    data_winsorized[col] = data_winsorized[col].clip(lower=lower_bound, upper=upper_bound)
    return data_winsorized

In [ ]:
df_winsorized = winsorize_outliers(df, '가격')

,주문번호,가격,수량,카테고리,지역,총금액
0,ORD0001,57451.0,5,가구,부산,287255
1,ORD0002,47926.0,5,식품,서울,239630
2,ORD0003,59715.0,4,식품,부산,238860
3,ORD0004,72845.0,7,도서,부산,509915
4,ORD0005,46488.0,3,식품,광주,139464
...,...,...,...,...,...,...
995,ORD0996,211943.0,88,식품,부산,18650984
996,ORD0997,282196.0,91,식품,서울,25679836
997,ORD0998,318148.4,67,식품,인천,32608565
998,ORD0999,318148.4,72,도서,광주,34998480


# 혼자 연습 (transform 써보기)

In [92]:
import pandas as pd

df = pd.DataFrame({
    '부서': ['영업', '영업', '기획', '기획', '개발', '개발', '개발'],
    '이름': ['민정', '철수', '영희', '수지', '태호', '준수', '하나'],
    '성과': [100, 120, 90, 85, 130, 110, 140]
})

In [93]:
# 각 행에 부서 평균 성과 붙이고 싶다
df['부서평균'] = df.groupby('부서')['성과'].transform('mean')

# 성과비율 계산 (%)
df['성과비율'] = df['성과'] / df['부서평균'] * 100

# 부서 내 최고 성과자 여부 (True/False)
df['성과'] == df.groupby('부서')['성과'].transform('max')
print(df)

   부서  이름   성과        부서평균        성과비율
0  영업  민정  100  110.000000   90.909091
1  영업  철수  120  110.000000  109.090909
2  기획  영희   90   87.500000  102.857143
3  기획  수지   85   87.500000   97.142857
4  개발  태호  130  126.666667  102.631579
5  개발  준수  110  126.666667   86.842105
6  개발  하나  140  126.666667  110.526316


### 혼자 연습 (lambda)

In [ ]:
nums = [1, 2, 3, 4, 5]

# 여기서 lambda 사용해봐
# 결과: [1, 4, 9, 16, 25]

제곱 = list(map(lambda x:x**2, nums))
print(제곱)

In [ ]:
words = ['apple', 'banana', 'kiwi']

# 각 단어의 길이를 출력하기
print(list(map(lambda x:len(x), words)))

# 리스트 컴프리헨션
print([len(word) for word in words])

In [ ]:
import pandas as pd

df = pd.DataFrame({
    '이름': ['민정', '수', '태호', '라라', '정', '울랄라'],
    '나이': [23, 45, 34, 22, 40, 100]
})

# 이름이 3글자 이상이면 True, 아니면 False인 새로운 열 '이름길이_3이상'을 만들어봐.

df['이름길이_3이상'] = list(map(lambda x:len(x)>=3, df['이름']))

df['이름길이_3이상'] = df['이름'].apply(lambda x: len(x) >= 3)

In [ ]:
df = pd.DataFrame({
    '이름': ['민정', '수지', '태호', '기태', '정민'],
    '점수': [95, 70, 58, 88, 45]
})

# 점수가 90 이상이면 '우수', 60 이상이면 '보통', 아니면 '미달' 로 구분된 '등급' 컬럼을 lambda로 만들어봐.

df['등급'] = df['점수'].apply(lambda x: '우수' if x >=90 else ('보통' if x >=60 else '미달'))
print(df)


# 아니면 for문 돌려
등급리스트 = []

for num in df['점수']:
    if num >= 90:
        등급리스트.append('우수')
    elif num >= 60:
        등급리스트.append('보통')
    else:
        등급리스트.append('미달')

df['등급2'] = 등급리스트
df

In [ ]:
# 그룹별 최고점 여부 (lambda + groupby + transform)
# 위 데이터에서, 각 점수가 해당 이름의 첫 글자 기준으로 최고 점수인지 True/False 표시해봐.

df = pd.DataFrame({
    '이름': ['민정', '민수', '태호', '태연', '정수'],
    '점수': [95, 70, 88, 91, 85]
})

df['최고'] = df['점수'] == df.groupby(df['이름'].str[0])['점수'].transform(max)
df

# 힌트
# 이름의 첫 글자 기준으로 그룹 나누기
# 그룹별 최고점 구하고
# 원래 점수와 비교